# Notebook which visualises original trajectories and compares them to disk-hashed trajectories


In [17]:
import folium
import os
import sys

currentdir = os.path.dirname(os.path.abspath("__file__"))
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)

from hashed_similarities.disk_similarity import (
    generate_disk_hash_similarity_coordinates,
)
from constants import R_MAX_LAT, R_MIN_LAT, R_MAX_LON, R_MIN_LON, COLOR_MAP as colors

ROME_META_FILE = "../dataset/rome/output/META-50.txt"
RESOLUTION = 1.0
LAYERS = 2
DISKS = 30

In [18]:
# Initialize a Folium map at a central point
map = folium.Map(
    location=[(R_MAX_LAT + R_MIN_LAT) / 2, (R_MAX_LON + R_MIN_LON) / 2],
    zoom_start=13,
    tiles=None,
)

# Define the corners of the rectangle
corners = [
    (R_MAX_LAT, R_MAX_LON),
    (R_MAX_LAT, R_MIN_LON),
    (R_MIN_LAT, R_MIN_LON),
    (R_MIN_LAT, R_MAX_LON),
    (R_MAX_LAT, R_MAX_LON),
]

folium.Rectangle(
    bounds=[(R_MIN_LAT, R_MIN_LON), (R_MAX_LAT, R_MAX_LON)],
    color="black",
    opacity=0.5,
).add_to(map)


folium.TileLayer(
    tiles="https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png",
    attr='Map data © <a href="https://openstreetmap.org">OpenStreetMap</a> contributors',
    name="Transparent Tiles",
    opacity=0.2,
).add_to(map)

In [19]:
def read_coordinates(file_path):
    coordinates = []
    with open("../dataset/rome/output/" + file_path, "r") as file:
        count = 0
        for line in file:
            count += 1
            lat, lon = line.strip().split(", ")
            coordinates.append((float(lat), float(lon)))
        # print(f"Number of coordinates for {file}: {str(count)}")
    return coordinates


# Read the index file to get the list of files containing coordinates
with open(ROME_META_FILE, "r") as index_file:
    number_of_trajectories = 0

    for file_name in index_file:
        if number_of_trajectories == 2:
            break
        file_name = file_name.strip()  # Remove newline characters
        coordinates = read_coordinates(file_name)
        # Plot each coordinate on the map
        for lat, lon in coordinates:
            folium.CircleMarker([lat, lon], radius=1, color="black", fill=False).add_to(
                map
            )
        number_of_trajectories += 1


In [20]:
measure = "dtw"
hashed_coordinates, all_disks = generate_disk_hash_similarity_coordinates(
    city="rome", diameter=RESOLUTION, layers=LAYERS, disks=DISKS, measure=measure, size=50
)

print("Number of layers:", len(all_disks))
print("Disk diameter:", RESOLUTION, "km")
for layer in all_disks:
    # print(f"Number of disks in layer {layer}: {len(all_disks[layer])}")
    for disk in all_disks[layer]:
        lat, lon = disk
        # print(f"Disk \t{disk}")
        folium.Circle(
            [lat, lon], radius=RESOLUTION*1000/2, color=colors[layer], fill=False
        ).add_to(map)

print("\n")
number_of_trajectories = 0
for trajectory_key, trajectory_hash in hashed_coordinates.items():
    if number_of_trajectories == 2:
            break
    layer = 0
    for hashed_trajectory_layer in trajectory_hash:
        print(f"Length of hashed coordinate in layer {layer}: {len(hashed_trajectory_layer)}")
        for hashed_coordinate in hashed_trajectory_layer:
            folium.Circle(
                hashed_coordinate, radius=RESOLUTION*1000/2, color=colors[layer], fill=True, fill_color=colors[layer]
            ).add_to(map)
        if len(hashed_trajectory_layer) > 1:
            folium.PolyLine(hashed_trajectory_layer, color=colors[layer],
                            weight=4, opacity=1).add_to(map)

        layer += 1
    number_of_trajectories += 1

map

Number of layers: 2
Disk diameter: 1.0 km


Length of hashed coordinate in layer 0: 4
Length of hashed coordinate in layer 1: 5
Length of hashed coordinate in layer 0: 4
Length of hashed coordinate in layer 1: 5
